In [1]:
%pylab --no-import-all
from os import path
import pandas as pd

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [2]:
try:
    file = path.join("..", "data", "interim", "london.csv")
except OSError:
    print("This repository does not host the data. "
          "Put the csv in ../data/interim/")
    raise
df = pd.read_csv(file, na_values=0)

Rename Italian columns to English.

In [3]:
df.rename(columns={'DONNA': 'ID',
                   'P_SPEZZ': 'GROUP_ID',
                   'P_CICLO': 'CYCLE_ID',
                   'ANNO_NAS': 'BIRTH_YR',
                   'DATA': 'BEGIN_DATE',
                   'T_SPEZZ': 'N_GROUPS',
                   'T_CICLI': 'N_CYCLES',
                   'QUALIFI': 'DESC',
                   'TIPOTEMP': 'TEMP_SCALE',
                   'L_CICLO': 'L_CYCLE',
                   'L_PREOV': 'L_PREOVULATION',
                   'L_PERIOD': 'L_PERIOD',
                   'FIGLI': 'CHILDREN'
                  },
         inplace=True)

We delete the cycles (rows) where the `L_PREOV` is missing (NA), because this data is unusable - we don't have the variable we seek to predict. **Ex:** If row 100 is deleted, the index would be $..., 98, 99, 101, 102, ...$

After these rows are dropped, there are holes created in the index of the data frame. We would like the index numbers to be consecutive, so we use the `reset_index` function.

In [5]:
df.dropna(subset=['L_PREOV'], inplace=True)

KeyError: ['L_PREOV']

Delete those where DESC != 1 because those are flawed entries.

In [10]:
df = df[df.DESC == 1]
df.drop('DESC', 1, inplace=True)

Convert all measurements to Fahrenheit.

In [11]:
for i in range(1, 100):
    df.ix[df.TEMP_SCALE != 1, 'TEMP' + str(i)] = 32 + 9 / 5 * df.ix[df.TEMP_SCALE != 1, 'TEMP' + str(i)]
df.drop(['TEMP_SCALE'], axis='columns', inplace=True)

In [ ]:
Create an age measurement for each cycle, computed from the birth year and measurement date.